# Obtaining movie data, API-testing

In [ ]:
# open questions:

# API only allows 1k requests per day..
# initial load (static database) or load on request, maybe another API required then?
# regular updates?

In [4]:
import requests
import pandas as pd

# get imdb ids

In [ ]:
# uses links.csv, a list of random imdbIds from https://grouplens.org/datasets/movielens/ ,  to obtain imdb ids, 
# then loops through some of them and puts them into a list

In [5]:
def get_ids(n):
    dtype_dic= {'movieId': str,'imdbId' : str, "tmdbId": str}
    IDdf = pd.read_csv("data/temp_links.csv", dtype = dtype_dic)
    #IMDB IDs to eventually be used as index
    idlist = list(IDdf["imdbId"].head(n))
    return idlist

In [6]:
imdbIDs = get_ids(500)
imdbIDs

['0114709',
 '0113497',
 '0113228',
 '0114885',
 '0113041',
 '0113277',
 '0114319',
 '0112302',
 '0114576',
 '0113189',
 '0112346',
 '0112896',
 '0112453',
 '0113987',
 '0112760',
 '0112641',
 '0114388',
 '0113101',
 '0112281',
 '0113845',
 '0113161',
 '0112722',
 '0112401',
 '0114168',
 '0113627',
 '0114057',
 '0114011',
 '0114117',
 '0112682',
 '0115012',
 '0112792',
 '0114746',
 '0112431',
 '0112818',
 '0113442',
 '0112697',
 '0112749',
 '0114279',
 '0112819',
 '0114272',
 '0113855',
 '0114681',
 '0113347',
 '0114369',
 '0114148',
 '0114916',
 '0114814',
 '0113819',
 '0110299',
 '0112499',
 '0113158',
 '0113321',
 '0110877',
 '0113419',
 '0116260',
 '0113862',
 '0116126',
 '0118002',
 '0115683',
 '0116839',
 '0113149',
 '0113118',
 '0116367',
 '0113010',
 '0113537',
 '0113828',
 '0115644',
 '0115676',
 '0114367',
 '0113973',
 '0112744',
 '0116731',
 '0112445',
 '0114660',
 '0112379',
 '0114039',
 '0112365',
 '0118158',
 '0116151',
 '0115697',
 '0113972',
 '0117002',
 '0114825',
 '01

# get data from omdb

In [ ]:
# http://www.omdbapi.com/
# API-key d3de5220
# max # of requests per day ~ 1k

In [49]:
# Receiving data from API and putting it into df
def get_data_from_omdb(imdbIDs):
    df0 = pd.DataFrame()
    for id in imdbIDs:
        url = f"http://www.omdbapi.com/?i=tt{id}&apikey=d3de5220"
        result = requests.get(url)
        j = result.json()
        df_single_movie = pd.DataFrame(j)
        df0 = pd.concat([df0, df_single_movie])
    return df0


def perform_cleaning(df):
    
    # turns date of release into date format 
    df["Released"] = pd.to_datetime(df["Released"])
    
    #converting "xx mins" into "xx"
    def get_mins(x):
        y = x.replace(" min", "")
        return y
    df["Runtime"] = df["Runtime"].apply(get_mins) 
    df["Runtime"] = pd.to_numeric(df["Runtime"])
    
    # drops duplicates, for some reason same movie appears always three times in df when converting json file...
    df0 = df.drop_duplicates("imdbID", keep = "first", inplace = False)
    return df0

In [50]:
df_raw = get_data_from_omdb(imdbIDs)

In [51]:
df = df_raw.copy()
df = perform_cleaning(df_raw)

In [52]:
df.to_csv("data/OMDB.csv", index = False)

# "parked" code for now

In [20]:
#df0 = pd.read_csv("data/OMDB.csv")
#df0.info()

In [58]:
#df = pd.read_csv("data/OMDB.csv", dtype = df_dtypes)
#df.head(3)

In [45]:
# provide a list of datatypes that the columns shall have --> leading zeros?
'''
df_columns = ["Title", 'Year', 'Rated', 'Released', 'Runtime', 'Genre', 'Director','Writer','Actors','Plot','Language', \
              'Country','Awards', 'Poster', 'Ratings','Metascore','imdbRating','imdbVotes','imdbID','Type','DVD',\
              'BoxOffice','Production','Website','Response']

df_dtypes = {'Title': str,'Year' : int, "Rated": str, "Released" : str, "Runtime": int, "Genre": str, "Director": str, \
            "Writer": str, "Actors": str, "Plot": str, "Language": str, "Country": str, "Awards": str, "Poster": str, \
            "Ratings": str, "Metascore": int, "imdbRating": str, "imdbVotes": str, "imdbID": str, "Type": str, \
            "DVD": str, "BoxOffice": str, "Production": str, "Website": str, "Response": str}
'''